In [1]:
import os
import time

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

from pyspark.sql.functions import col

from pyspark.mllib.recommendation import ALS, Rating
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics

In [2]:
books = spark.read.load(r"../Dataset/BX-Books.csv",
                     format="csv", sep=";", inferSchema="true", header="true", encoding ='latin1')

In [3]:
win_row_number = Window.orderBy("ISBN")
isbn = books.select(F.row_number().over(win_row_number).alias("book_id"), "ISBN")

In [4]:
ratings = spark.read.load(r"../Dataset/BX-Book-Ratings.csv",
                     format="csv", sep=";", inferSchema="true", header="true", encoding ='latin1')

In [5]:
ratings = ratings.withColumn("Book-Rating", ratings["Book-Rating"].cast("Float"))
ratings_with_bid = ratings.join(isbn, on=["ISBN"], how="inner").select(col("User-ID").alias("user"), col("book_id").alias("item"), col("Book-Rating").alias("rating"))

In [6]:
training, test = ratings_with_bid.randomSplit([0.8, 0.2], 24)

In [7]:
training.show()

+----+------+------+
|user|  item|rating|
+----+------+------+
| 432|132685|   0.0|
| 433|190910|   5.0|
| 507| 83084|   0.0|
| 850|254896|   9.0|
|1131| 23638|   8.0|
|1706| 93714|   7.0|
|1848|119715|   0.0|
|1903|195798|   0.0|
|2033|202687|  10.0|
|2276|112560|   0.0|
|2276|134856|   0.0|
|2545|260897|   0.0|
|2770|190910|   6.0|
|2793|109232|   0.0|
|2793|170183|   0.0|
|3193|155947|   5.0|
|3329| 42081|   7.0|
|3363| 93255|   0.0|
|3363| 93651|   0.0|
|3363|118660|   0.0|
+----+------+------+
only showing top 20 rows



### Hyperparameters
<ul>
<li><b>Rank</b> : 6</li>
<li><b>Iterations</b>: 100</li>
<li><b>reg</b> : 0.01</li>
</ul>

In [18]:
model = ALS.train(training, rank=6, iterations=100, lambda_=0.01, blocks=-1, nonnegative=False, seed=None)

Py4JJavaError: An error occurred while calling o255.trainALSModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 635.0 failed 1 times, most recent failure: Lost task 0.0 in stage 635.0 (TID 12289, localhost, executor driver): java.lang.StackOverflowError
	at java.io.ObjectInputStream$PeekInputStream.read(ObjectInputStream.java:2661)
	at java.io.ObjectInputStream$BlockDataInputStream.read(ObjectInputStream.java:3078)
	at java.io.ObjectInputStream$BlockDataInputStream.readFully(ObjectInputStream.java:3102)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2274)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2209)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2067)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1571)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2285)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2209)


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 43548)
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/anaconda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/anaconda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/anaconda/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/opt/spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/opt/spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/opt/spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py", line 245, in accum_updat

In [23]:
model = ALS.train(training, rank=10, iterations=30, lambda_=0.3, blocks=-1, nonnegative=True, seed=None)

Py4JJavaError: An error occurred while calling o308.trainALSModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 934.0 failed 1 times, most recent failure: Lost task 3.0 in stage 934.0 (TID 16832, localhost, executor driver): java.lang.StackOverflowError
	at java.io.ObjectInputStream.readHandle(ObjectInputStream.java:1682)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1743)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2040)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1571)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2285)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2209)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2067)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1571)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2285)
	at java.io.ObjectInputStream.readSer

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 57417)
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/anaconda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/anaconda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/anaconda/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/opt/spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/opt/spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/opt/spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py", line 245, in accum_updat

In [10]:
test.show(3)

+----+------+------+
|user|  item|rating|
+----+------+------+
| 254| 99105|   0.0|
| 507| 82280|   7.0|
|2358|147113|   0.0|
+----+------+------+
only showing top 3 rows



In [15]:
predicted = model.predictAll(test.select('user', 'item').rdd)

In [16]:
predicted_df = predicted.toDF().select(col("user"), col("product").alias("item"), col("rating").alias("prediction"))

In [18]:
preformance_df = test.join(predicted_df, on=['user', 'item'], how='inner')

In [22]:
preformance_df.show(50)

+-----+------+------+-------------------+
| user|  item|rating|         prediction|
+-----+------+------+-------------------+
|    9|103718|   6.0|                0.0|
|  243| 45044|   0.0| 1.0045305102697282|
|  507|122295|   0.0|0.06935991861669599|
| 2033| 89538|   7.0|  2.840559944205026|
| 2276|141397|   9.0|  6.723893642586248|
| 2288| 34015|   0.0|  5.509460728809108|
| 2977|135472|  10.0| 0.2049334692214977|
| 5499| 34238|  10.0| 12.693202412809335|
| 5582| 42352|   7.0|  10.09994501897297|
| 6251|  1486|   8.0|                0.0|
| 6323|121498|   0.0|                0.0|
| 6543|121823|   0.0|0.09424125002711214|
| 6575|144559|   0.0| 0.0689910888269809|
| 6731| 67931|   7.0| 1.0604564424414225|
| 7210|197681|   0.0| 1.8151706114337132|
| 8245| 21514|   0.0|   3.32826961875425|
| 8253| 96675|  10.0|  4.572258785223532|
| 8362| 86663|   0.0| 2.0128465049948234|
| 8936|  9999|   9.0|                0.0|
| 9094| 91541|   0.0|  7.802372303967748|
|10030|147457|   0.0| 3.5643307644

In [19]:
metric = RegressionMetrics(preformance_df.select('rating', 'prediction').rdd)

In [20]:
metric.rootMeanSquaredError

4.030247373533029